## Weighted RNN for News_Text Classification

### Importing Requured Libraries

In [12]:
import os
import sys
import pickle
import keras
import numpy as np
import pandas as pd
import numpy
import matplotlib as plt
%matplotlib inline

import gensim 
from gensim.models import Word2Vec, KeyedVectors
from gensim.test.utils import common_texts, get_tmpfile

from numpy import loadtxt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import text
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import  Dense, Input, LSTM, Embedding, Dropout, Activation ,Permute ,SimpleRNN , Bidirectional ,TimeDistributed, Conv1D ,Activation, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import sklearn.metrics as metrics


### Loading Test/Train Data

In [2]:
# Loading traning and Testing data

# To read from pickled file
with open('train_data.p', 'rb') as fp:
    train_data = pickle.load(fp)
with open('desired_train_data.p', 'rb') as fp:
    train_y = pickle.load(fp)
with open('test_data.p', 'rb') as fp:
    test_data = pickle.load(fp)
with open('desired_test_data.p', 'rb') as fp:
    test_y = pickle.load(fp)

In [3]:
print(len(train_data))
print(len(train_y))
print(len(test_data))
print(len(test_y))


11314
11314
7532
7532


In [4]:

wv = KeyedVectors.load('vectors.kv', mmap='r')
#print(wv.key_to_index)
#print(wv.index_to_key)
print(wv)

KeyedVectors<vector_size=200, 43361 keys>


In [5]:
print(len(wv['the']))
#print(wv['the'])

200


## Implementation of W_RNN

### a)Text length selection

In [6]:
DLs = [] # length of each document
docs = train_data + test_data
#print(len(docs))
for i in docs:
    count = 0
    for j in i:
        count += 1
    DLs.append(count)
    
#print(len(DLs))
#print(DLs[0])

DNum = len(docs)
print(DNum)
DLs.sort()
#print(DLs)

DNum_2 = 0
SL = 400
for i in DLs:
    if i < SL :
       DNum_2 += 1 

theta = (DNum_2 / DNum)*100
print(theta)

18846
84.89334606813117


In [7]:
#Truncation and dealing with text smaller than SL

#Train data
data_train = []
zero_wv = [0]*200
for doc in train_data:
    doc_wv = []
    count = 0
    for j in doc:
        count += 1
        doc_wv.append(wv[j])
        if count == 400:
            break
    if (len(doc_wv) != 400):
        while (len(doc_wv) != 400):
            doc_wv.append(zero_wv)
    data_train.append(doc_wv)

#Test data
data_test = []
zero_wv = [0]*200
for doc in test_data:
    doc_wv = []
    count = 0
    for j in doc:
        count += 1
        doc_wv.append(wv[j])
        if count == 400:
            break
    if (len(doc_wv) != 400):
        while (len(doc_wv) != 400):
            doc_wv.append(zero_wv)
    data_test.append(doc_wv)


In [8]:
print(len(data_train), len(data_test))
print(len(train_y), len(test_y))


11314 7532
11314 7532


In [9]:
#Finally

#To create required input for the Keras Model
truncate = len(data_train)-(len(data_train)%128)
print(truncate)
data_train = data_train[:truncate]
train_y = train_y[:truncate]

truncate = len(data_test)-(len(data_test)%128)
print(truncate)

data_test = data_test[:truncate]
test_y = test_y[:truncate]

11264
7424


###### list = [1,1,2,4,5,6,7,5,6,7,3,5,7,2,2,4,6]
###### print(len(list))
###### t = 10
###### list = list[4:t]
###### print(list)

In [10]:
data_train = np.array(data_train)
y_train = np.array(train_y)
data_test = np.array(data_test)
y_test = np.array(test_y)



In [11]:
print(data_train.shape, data_test.shape , y_train.shape, y_test.shape)


(11264, 400, 200) (7424, 400, 200) (11264, 20) (7424, 20)


In [12]:
# Debugging # y_test is one hot encoded for 20 categories
for i in y_train:
    if list(i) == [0]*20:
        print('Wrong')


In [13]:
# Backup
'''
import copy
backup_data_train = copy.deepcopy(data_train)
backup_y_train = copy.deepcopy(y_train)
backup_data_test = copy.deepcopy(data_test)
backup_y_test = copy.deepcopy(y_test)
'''

'\nimport copy\nbackup_data_train = copy.deepcopy(data_train)\nbackup_y_train = copy.deepcopy(y_train)\nbackup_data_test = copy.deepcopy(data_test)\nbackup_y_test = copy.deepcopy(y_test)\n'

In [14]:
# Shuffle two nimpy arrays together
#arr1 = np.array([[1, 2], [3, 4], [5, 6]])
#arr2 = np.array([7, 8, 9])

def shuffle_in_unison(a,b): #Inplace
    #only shuffles along the first axis of a multi-dimensional array.
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b) 
    return a,b

def shuffle_in_unison1(a,b): # Using Temp Extra Memory
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [15]:
print(len(data_train),len(y_train),len(y_test),len(data_test))

11264 11264 7424 7424


In [16]:
# Final Shuffled Arrays
data_train,y_train = shuffle_in_unison(data_train,y_train)
data_test, y_test = shuffle_in_unison(data_test,y_test)

In [17]:
# We have trianing and testing data which can be fed to the network now

## Helper Functions

In [18]:
# Saving a keras model
from numpy import loadtxt
# model is name of the model to be save , file_name is file with which to save
def mysave(model,file_name): # in file_name include .h5
    model.save(file_name)


In [19]:
# loading a saved keras model
def myload(file_name): # in file_name include .h5
    # load model
    model = load_model(file_name)
    # summarize model.
    model.summary()
    return model

## Models

### SimpleRNN(Reference Model-I)

In [20]:
# Constants
batch_Size = 128
adam = keras.optimizers.Adam(learning_rate=0.01)
early = EarlyStopping(monitor = 'val_categorical_accuracy', verbose=1, mode = 'max', patience = 3)

In [ ]:
## Network architecture
#model name - modelrnn, history name - history_rnn
modelrnn = Sequential()
modelrnn.add(LSTM(128, input_shape=(400,200), dropout=0.2,recurrent_dropout=0.2,return_sequences = False))
modelrnn.add(Dense(128,activation='relu'))
#modelrnn.add(Dense(128,activation='relu'))
modelrnn.add(Dense(20,activation='softmax'))
modelrnn.compile(loss='categorical_crossentropy', optimizer=adam, metrics=[keras.metrics.categorical_accuracy])
## Fit the model
history_rnn = modelrnn.fit(data_train, y_train, validation_split=0.01, epochs=10 ,batch_size=batch_Size)
modelrnn.summary()

/home/helia/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
